# CONSIGNA 1:

**_1) Cargamos los datos_**
- Carguen el dataset:
```
  data = pd.read_csv("encuesta-anual-hogares-2019.csv", sep=',') 
```
**_2) Inspección inicial_**
- Eliminen las columnas `id` y `hijos_nacidos_vivos`

**_3) Discretización_**
- Para las siguientes columnas discreticen por igual frecuencia e igual rango.
    <br>`ingresos_familiares` con q=8
    <br>`ingreso_per_capita_familiar` con q=10

- En algunas situaciones hay ciertos elementos que se repiten al momento de discretizar, una forma de eliminar duplicados es con el argumento `duplicates='drop'`.
    <br><br>Para las siguientes columnas `ingreso_total_lab` y `ingreso_total_no_lab` consideren:
    ```
    data['ingreso_total_lab'] = pd.qcut(data['ingreso_total_lab'], q=10, duplicates='drop')
    data['ingreso_total_no_lab'] = pd.qcut(data['ingreso_total_no_lab'], q=4, duplicates='drop')
    ```

- Para la columna `edad` discreticen usando igual distancia con `bins=5`.

**_4) Preparación de datos_**
- Cambien el tipo de dato a `str` de las siguientes columnas: `comuna` y `nhogar`.
  <br>_¿Por qué hacemos esto?_ Para estas columnas el número es simplemente una connotación, para representar una comuna por ejemplo pero no hay una relación numérica entre ellos.

- _¿Qué esperas como valor en la columna `años_escolaridad`?_ Número enteros pero no siempre es así, cada entidad o empresa tiene diferentes formas de rellenar una encuesta.
    <br>Evalua lo siguiente: `data['años_escolaridad'].unique()`, vas a poder ver los valores únicos en la columna. Donde destacamos que todos son `object/string`.

- Reemplazar `Ningun año de escolaridad aprobado` por un '0'. 
<br>Efectivamente por '0' y no 0, porque esta columna maneja datos tipo `object/string`.
    ```
    data['años_escolaridad'] = data['años_escolaridad'].replace('Ningun año de escolaridad aprobado', '0')
    ```

- Vamos a convertir los tipos de datos de la columna anterior `años_escolaridad` a enteros.
    <br>De manera intuitiva podríamos hacer:
    ```
    data['años_escolaridad'] = data['años_escolaridad'].astype(float).astype("Int32")
    ```
    PEROOOOOOO marca un error, ¿cierto?
    
    Posiblemente muchas veces les pase que cuando hagan una _cast_ (conversión de un tipo de dato a otro) pueden llegar a tener conflictos si esa columna tienen _NaN_. Para este caso si queremos convertir los valores de la columna `años_escolaridad` de _string_ a _int_, hay que hacer un paso intermedio que es pasarlo a _float_.
    ```
    data['años_escolaridad'] = data['años_escolaridad'].astype(float).astype("Int32")
    ```

- Discreticen para la columna `años_escolaridad` por igual frecuencia e igual rango con un `q=5`

- No necesariamente siempre hay que rellenar los `NaN` en todas las columnas, porque quizás esa cantidad de `NaN` no es tan representativa para nuestro análisis. Así que podes eliminarlo para todo el dataframe o para ciertas columnas.
    ```
    # Eliminar filas que contengan NaN
    data = data.dropna(subset=['situacion_conyugal', 'sector_educativo', 'lugar_nacimiento', 'afiliacion_salud'])
    ```
- Después de eliminar filas, podes resetear el índice para que mantenga la secuencia:
    ```
    data = data.reset_index(drop=True)
    ```

- Rellenar los datos faltantes para la columna `años_escolaridad`. Primero añadan la categoría `desconocido` y luego hacen un rellenado de los datos faltantes con `desconocido`.

- Rellenar los datos faltantes para la columna `nivel_max_educativo` con `value=desconocido`

In [ ]:
import pandas as pd
from funpymodeling.exploratory import freq_tbl, status, profiling_num

pd.set_option('display.max_columns', None)

data = pd.read_csv("./data/encuesta-anual-hogares-2019.csv", sep=",", encoding='ISO-8859-1') 
data_original = data.copy()

data.drop(['id', 'hijos_nacidos_vivos'], axis=1, inplace=True)

status(data)

In [ ]:
data['ingresos_familiares'], seved_bins_ingresos_familiares = pd.qcut(data['ingresos_familiares'], q=8, retbins=True)
data['ingreso_per_capita_familiar'], seved_bins_ingresos_familiares = pd.qcut(data['ingreso_per_capita_familiar'], q=8, retbins=True)
data['edad'], saved_bins_age = pd.cut(data['edad'], bins=5, retbins=True )


In [ ]:
data.hist(figsize=(10,10))

In [ ]:
profiling_num(data)

In [ ]:
data['ingreso_total_lab'] = pd.qcut(data['ingreso_total_lab'], q=10, duplicates='drop')
data['ingreso_total_no_lab'] = pd.qcut(data['ingreso_total_no_lab'], q=4, duplicates='drop')

freq_tbl(data['ingreso_total_no_lab'])



In [ ]:
data['comuna'] = data['comuna'].astype(str)
data['nhogar'] = data['nhogar'].astype(str)


In [ ]:
data.replace("Ningun año de escolaridad aprobado", "0", inplace=True)
data['años_escolaridad'] = data['años_escolaridad'].astype(float).astype('Int16')
data['años_escolaridad'].fillna(0, inplace=True)
data['años_escolaridad'], saved_bins_anos_escolaridad = pd.qcut(data['años_escolaridad'], q=5, retbins=True)

In [ ]:
data = data.dropna(subset=['situacion_conyugal', 'sector_educativo', 'lugar_nacimiento', 'afiliacion_salud'])
data.reset_index(drop=True, inplace=True)

In [ ]:
data['nivel_max_educativo'] = data['nivel_max_educativo'].astype('category')
data['nivel_max_educativo'] = data['nivel_max_educativo'].cat.add_categories('desconocido')
data['nivel_max_educativo'].fillna('desconocido', inplace=True)

status(data)

# CONSIGNA 2:

**_5) One hot encoding_**

- Hagan `data_ohe = pd.get_dummies(data)`
- Guardar `data_ohe` en un archivo pickle como vimos en clase con el nombre `categories_ohe.pickle`.
- Carguen el dataset `new_data = pd.read_csv("new_data.csv", sep=',')`
- A `new_data` hagan un reindex con las columnas que guardaron el archivo pickle y para los valores `NaN` rellenenlos con un `0`.

In [ ]:
data_ohe = pd.get_dummies(data)
data_ohe

In [ ]:
import pickle

with open('./data/categories_ohe.pickle', 'wb') as handle:
    pickle.dump(data_ohe.columns, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
new_data = pd.read_csv("./data/new_data.csv", sep=",", encoding='ISO-8859-1') 
with open('./data/categories_ohe.pickle', 'rb') as handle:
    ohe_tr = pickle.load(handle)

new_data = pd.get_dummies(new_data).reindex(columns = ohe_tr, fill_value=0)
new_data

## 📍 Consigna 3

Cargar su notebook y datasets a un repositorio público personal y compartirlo por Discord.
<br>Consideren usar git lfs para los dataset con extensión csv.